In [71]:
# 1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
# Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
# Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp 
# для данного значения userId.

import pandas as pd
rating = pd.read_csv('ratings.csv')
#rating.head()
#создаем таблицу с пользователями более 100 оценок

rating_count = rating[['userId','rating']].groupby('userId').count().reset_index()
rating_count.head()
rating_more_100 = rating_count[rating_count['rating']>100]
rating_more_100.head()

#создаем таблицу с минимальными и максимальными значениями столбца timestamp

rating_max = rating[['userId','timestamp']].groupby('userId').max().reset_index()
rating_max = rating_max.rename(columns = {'timestamp': 'max_timestamp'})
rating_min = rating[['userId','timestamp']].groupby('userId').min().reset_index()
rating_min = rating_min.rename(columns = {'timestamp': 'min_timestamp'})
#объединяем 3 таблицы

life_user = rating_more_100.merge(rating_max, how ='outer').merge(rating_min, how ='outer')
#life_user.head()
#считаем разницу между столбцами timestamp
def diff (timestamp):
    mean =  (timestamp['max_timestamp'] - timestamp['min_timestamp'])
    return  mean      

life_user['diff'] = life_user.apply(diff, axis = 1)
life_user.head()


,userId,rating,max_timestamp,min_timestamp,diff
0,1,232.0,965719662,964980499,739163.0
1,4,216.0,1007574542,945078428,62496114.0
2,6,314.0,845556915,845553109,3806.0
3,7,152.0,1176181731,1106635416,69546315.0
4,10,140.0,1455619275,1455301553,317722.0


In [101]:
#2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать
#две таблицы:
#- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
#- аналогичную таблицу по типам выручки с указанием адреса клиента
#Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

#объединяем все таблицы , включая адреса
client_base_1 = client_base.merge(rzd, how ='outer', on='client_id').\
    merge(auto,how ='outer', on='client_id').\
    merge(air,how ='outer', on='client_id')
client_base_1.fillna(0)

#объединяем все таблицы, исключая адрес
client_base_no_address = client_base_1[['client_id', 'rzd_revenue', 'auto_revenue', 'air_revenue']]
client_base_no_address.fillna(0)

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [ ]:
3.В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id 
известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 
Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего
потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится
на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что 
дают.

In [ ]:
координаты можно приобщить к идентификатору пользователя, если один пользователь "заходит" из разных мест;
координаты можно фильтровать по наиболее посещаемым местам по координатам.
